# California 2Monthly Prediction

## Imports and Functions

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
from sklearn.gaussian_process import GaussianProcessRegressor


from sklearn.svm import SVR
import os
import numpy as np
from xgboost import XGBRegressor
import sys

In [2]:
# Add the folder to the Python path

os.chdir("../")
# change working directory to project's root path
print(os.getcwd())

FIRST_YEAR= 1972
FREQUENCY= "2monthly"

c:\Users\marti\Desktop\data\hw_extra


In [3]:
folder_path = os.path.abspath("functions/") #INPUT_PATH)#'path_to_your_folder')  # Replace with the actual folder path
sys.path.insert(0, folder_path)

from Predictions import (
    PredictionExperiment,
    PredictionModel
)

In [13]:
region="california"
metadata = pd.read_csv(f"data/new_features_t2m/{region}/metadata.csv")
metadata = metadata[metadata["frequency"]==FREQUENCY]
metadata.reset_index(inplace=True, drop=True)
display(metadata)

,id,filename,boxes,top_n,var_thresh,modes,frequency,season,methods,extra_indices,only_sea
0,dc3bd026,predictor_dc3bd026_1.parquet,0,5,NaN,1,2monthly,1,ps,NaN,False
1,dc3bd026,predictor_dc3bd026_2.parquet,0,5,NaN,1,2monthly,2,ps,NaN,False
2,dc3bd026,predictor_dc3bd026_3.parquet,0,5,NaN,1,2monthly,3,ps,NaN,False
3,dc3bd026,predictor_dc3bd026_4.parquet,0,5,NaN,1,2monthly,4,ps,NaN,False
4,dc3bd026,predictor_dc3bd026_5.parquet,0,5,NaN,1,2monthly,5,ps,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...
1435,f8698b6f,predictor_f8698b6f_8.parquet,0,30,0.15,3,2monthly,8,ps,PDO-ONI-SAM,False
1436,f8698b6f,predictor_f8698b6f_9.parquet,0,30,0.15,3,2monthly,9,ps,PDO-ONI-SAM,False
1437,f8698b6f,predictor_f8698b6f_10.parquet,0,30,0.15,3,2monthly,10,ps,PDO-ONI-SAM,False
1438,f8698b6f,predictor_f8698b6f_11.parquet,0,30,0.15,3,2monthly,11,ps,PDO-ONI-SAM,False


In [17]:
last_id = "f2406afd"
index = metadata[metadata["id"]==last_id]
metadata = metadata.iloc[list(index.index)[-1]+1:]
metadata

,id,filename,boxes,top_n,var_thresh,modes,frequency,season,methods,extra_indices,only_sea
972,3a03ff61,predictor_3a03ff61_1.parquet,0,10,NaN,2,2monthly,1,ps,PDO-ONI-SAM,False
973,3a03ff61,predictor_3a03ff61_2.parquet,0,10,NaN,2,2monthly,2,ps,PDO-ONI-SAM,False
974,3a03ff61,predictor_3a03ff61_3.parquet,0,10,NaN,2,2monthly,3,ps,PDO-ONI-SAM,False
975,3a03ff61,predictor_3a03ff61_4.parquet,0,10,NaN,2,2monthly,4,ps,PDO-ONI-SAM,False
976,3a03ff61,predictor_3a03ff61_5.parquet,0,10,NaN,2,2monthly,5,ps,PDO-ONI-SAM,False
...,...,...,...,...,...,...,...,...,...,...,...
1435,f8698b6f,predictor_f8698b6f_8.parquet,0,30,0.15,3,2monthly,8,ps,PDO-ONI-SAM,False
1436,f8698b6f,predictor_f8698b6f_9.parquet,0,30,0.15,3,2monthly,9,ps,PDO-ONI-SAM,False
1437,f8698b6f,predictor_f8698b6f_10.parquet,0,30,0.15,3,2monthly,10,ps,PDO-ONI-SAM,False
1438,f8698b6f,predictor_f8698b6f_11.parquet,0,30,0.15,3,2monthly,11,ps,PDO-ONI-SAM,False


In [18]:
kernel = RBF(length_scale=1.0) + WhiteKernel(noise_level=1)
regressors = [LinearRegression(),RandomForestRegressor(random_state=42, n_estimators=5), RandomForestRegressor(random_state=42, n_estimators=10), SVR(kernel='rbf'),
                XGBRegressor(random_state=42, n_estimators=10, learning_rate=0.1),XGBRegressor(random_state=42, n_estimators=15, learning_rate=0.1), GaussianProcessRegressor(kernel=kernel, random_state=42, n_restarts_optimizer=10)]
name_regressors = ["Linear", "RF5", "RF10", "SVR-rbf", "XGB10", "XGB15", "GPR"]
indices_of_interest = ["HWN", "HWF", "HWD", "HWM", "HWA"]

In [20]:
id_experiments = metadata["id"].unique()
for id in id_experiments:
    data = {i: pd.read_parquet(f"data/new_features_t2m/{region}/predictor_{id}_{i}.parquet") for i in range(1,13)}
    experiment_1 = PredictionExperiment(data, indices_of_interest, regressors, name_regressors, 5, id)
    experiment_1.execute_experiment()
    experiment_1.get_metrics("r2", "prediction", show=False)
    experiment_1.get_metrics("mape", "prediction", show=False)
    experiment_1.get_metrics("r2", stage="training", show=False)
    experiment_1.get_metrics("mape", stage="training", show=False)
    experiment_1.get_metrics("r2", stage="CV", show=False)
    experiment_1.get_metrics("mape", stage="CV", show=False)
    #experiment_1.top_results("r2", 5, stage="prediction", top_data_path=f"data/results/{FREQUENCY}/top_results.csv")
    #experiment_1.top_results("cv_r2", 5, stage="CV", top_data_path=f"data/results/{FREQUENCY}/top_results.csv")
    experiment_1.save_results(f"data/{region}_results/results.csv")

c:\Users\marti\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\gaussian_process\kernels.py:429: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
c:\Users\marti\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\gaussian_process\kernels.py:429: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
c:\Users\marti\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\gaussian_process\kernels.py:429: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warn

# Chile 2Monthly Prediction

In [4]:
region="chile"

In [5]:
metadata = pd.read_csv(f"data/new_features_t2m/{region}/metadata.csv")
metadata = metadata[metadata["frequency"]==FREQUENCY]
metadata.reset_index(drop=True,inplace=True)
metadata

,id,filename,boxes,top_n,var_thresh,modes,frequency,season,methods,extra_indices,only_sea
0,676a31ca,predictor_676a31ca_1.parquet,0,5,NaN,1,2monthly,1,ps,NaN,False
1,676a31ca,predictor_676a31ca_2.parquet,0,5,NaN,1,2monthly,2,ps,NaN,False
2,676a31ca,predictor_676a31ca_3.parquet,0,5,NaN,1,2monthly,3,ps,NaN,False
3,676a31ca,predictor_676a31ca_4.parquet,0,5,NaN,1,2monthly,4,ps,NaN,False
4,676a31ca,predictor_676a31ca_5.parquet,0,5,NaN,1,2monthly,5,ps,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...
1435,a59026f4,predictor_a59026f4_8.parquet,0,30,0.15,3,2monthly,8,ps,PDO-ONI-SAM,False
1436,a59026f4,predictor_a59026f4_9.parquet,0,30,0.15,3,2monthly,9,ps,PDO-ONI-SAM,False
1437,a59026f4,predictor_a59026f4_10.parquet,0,30,0.15,3,2monthly,10,ps,PDO-ONI-SAM,False
1438,a59026f4,predictor_a59026f4_11.parquet,0,30,0.15,3,2monthly,11,ps,PDO-ONI-SAM,False


In [6]:
last_id = "a3dcae44"
index = metadata[metadata["id"]==last_id]
metadata = metadata.iloc[list(index.index)[-1]+1:]
metadata

,id,filename,boxes,top_n,var_thresh,modes,frequency,season,methods,extra_indices,only_sea
672,af89d45c,predictor_af89d45c_1.parquet,0,10,0.15,3,2monthly,1,ps,NaN,False
673,af89d45c,predictor_af89d45c_2.parquet,0,10,0.15,3,2monthly,2,ps,NaN,False
674,af89d45c,predictor_af89d45c_3.parquet,0,10,0.15,3,2monthly,3,ps,NaN,False
675,af89d45c,predictor_af89d45c_4.parquet,0,10,0.15,3,2monthly,4,ps,NaN,False
676,af89d45c,predictor_af89d45c_5.parquet,0,10,0.15,3,2monthly,5,ps,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...
1435,a59026f4,predictor_a59026f4_8.parquet,0,30,0.15,3,2monthly,8,ps,PDO-ONI-SAM,False
1436,a59026f4,predictor_a59026f4_9.parquet,0,30,0.15,3,2monthly,9,ps,PDO-ONI-SAM,False
1437,a59026f4,predictor_a59026f4_10.parquet,0,30,0.15,3,2monthly,10,ps,PDO-ONI-SAM,False
1438,a59026f4,predictor_a59026f4_11.parquet,0,30,0.15,3,2monthly,11,ps,PDO-ONI-SAM,False


In [7]:
kernel = RBF(length_scale=1.0) + WhiteKernel(noise_level=1)
regressors = [LinearRegression(),RandomForestRegressor(random_state=42, n_estimators=5), RandomForestRegressor(random_state=42, n_estimators=10), SVR(kernel='rbf'),
                XGBRegressor(random_state=42, n_estimators=10, learning_rate=0.1),XGBRegressor(random_state=42, n_estimators=15, learning_rate=0.1), GaussianProcessRegressor(kernel=kernel, random_state=42, n_restarts_optimizer=10)]
name_regressors = ["Linear", "RF5", "RF10", "SVR-rbf", "XGB10", "XGB15", "GPR"]
indices_of_interest = ["HWN", "HWF", "HWD", "HWM", "HWA"]

In [8]:
id_experiments = metadata["id"].unique()
for id in id_experiments:
    data = {i: pd.read_parquet(f"data/new_features_t2m/{region}/predictor_{id}_{i}.parquet") for i in range(1,13)}
    experiment_1 = PredictionExperiment(data, indices_of_interest, regressors, name_regressors, 5, id)
    experiment_1.execute_experiment()
    experiment_1.get_metrics("r2", "prediction", show=False)
    experiment_1.get_metrics("mape", "prediction", show=False)
    experiment_1.get_metrics("r2", stage="training", show=False)
    experiment_1.get_metrics("mape", stage="training", show=False)
    experiment_1.get_metrics("r2", stage="CV", show=False)
    experiment_1.get_metrics("mape", stage="CV", show=False)
    #experiment_1.top_results("r2", 5, stage="prediction", top_data_path=f"data/results/{FREQUENCY}/top_results.csv")
    #experiment_1.top_results("cv_r2", 5, stage="CV", top_data_path=f"data/results/{FREQUENCY}/top_results.csv")
    experiment_1.save_results(f"data/{region}_results/results.csv")

c:\Users\marti\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\gaussian_process\kernels.py:429: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
c:\Users\marti\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\gaussian_process\kernels.py:429: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
c:\Users\marti\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\gaussian_process\kernels.py:429: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warn